In [11]:
import os
import cv2
import numpy as np
from shutil import copyfile
import random

def add_gaussian_blur(image, ksize=(5, 5), sigmaX=0):
    """
    이미지에 Gaussian Blur를 추가합니다.

    Parameters:
    image (numpy.ndarray): 원본 이미지
    ksize (tuple): Gaussian 커널의 크기
    sigmaX (float): Gaussian 커널의 표준 편차

    Returns:
    numpy.ndarray: 블러가 추가된 이미지
    """
    blurred_image = cv2.GaussianBlur(image, ksize, sigmaX)
    return blurred_image

def augment_with_gaussian_blur(input_folder, output_folder, ksize=(5, 5), sigmaX=0, blur_ratio=0.65):
    os.makedirs(output_folder, exist_ok=True)
    
    # 입력 폴더의 모든 이미지 파일을 가져오기
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    # 전체 이미지 수와 블러 처리할 이미지 수 계산
    total_images = len(image_files)
    blur_count = int(total_images * blur_ratio)
    
    # 블러 처리할 이미지를 랜덤하게 선택
    blur_images = random.sample(image_files, blur_count)
    
    for filename in image_files:
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        
        if img is not None:
            # 원본 이미지 복사
            copyfile(img_path, os.path.join(output_folder, filename))
            
            # 블러 처리할 이미지인 경우 블러 이미지 생성
            if filename in blur_images:
                blurred_img = add_gaussian_blur(img, ksize, sigmaX)
                blurred_filename = f"{os.path.splitext(filename)[0]}_blur{os.path.splitext(filename)[1]}"
                output_path = os.path.join(output_folder, blurred_filename)
                cv2.imwrite(output_path, blurred_img)
    
    return blur_images

def duplicate_labels(input_folder, output_folder, blur_images):
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):  # Assuming label files have .txt extension
            label_path = os.path.join(input_folder, filename)
            
            # 원본 파일 복사
            copyfile(label_path, os.path.join(output_folder, filename))
            
            # 블러 처리된 이미지의 라벨 복사
            base_filename = os.path.splitext(filename)[0]
            if f"{base_filename}.jpg" in blur_images or f"{base_filename}.png" in blur_images:
                noisy_filename = f"{base_filename}_blur{os.path.splitext(filename)[1]}"
                copyfile(label_path, os.path.join(output_folder, noisy_filename))

# 사용 예제
input_image_folder = '../base/images/train'
output_image_folder = './images/train'
blur_images = augment_with_gaussian_blur(input_image_folder, output_image_folder, ksize=(5, 5), sigmaX=0, blur_ratio=0.3)

input_label_folder = '../base/labels/train'
output_label_folder = './labels/train'
duplicate_labels(input_label_folder, output_label_folder, blur_images)

: 